<a href="https://colab.research.google.com/github/Migol123/K-IMU/blob/main/k-IMU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorboard
!pip install stable-baselines3
!pip install gymnasium
!pip install shimmy
!pip install torch
!pip install stable-baselines3 gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
import pandas as pd

#Ubicacion de instalaciones

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1jmwHTZZQeq6GykdGES0MEd8P1RuVwyzo/edit#gid=1920166751'
df_fac = pd.read_csv(f"{url.replace('/edit#gid=', '/export?format=csv&gid=')}")
df_fac

In [ ]:
import geopandas as gpd
from shapely.geometry import shape

#Puntos de demanda

In [ ]:
deman = gpd.read_file('https://drive.google.com/uc?id=1isYwg_uP5p4PpNcbHYjeU9CN6kLOrkmd')
deman=deman[deman['distrito'] == 'ALTO SELVA ALEGRE'].drop(['OBJECTID_1', 'departam','provinci','distrito','nomccpp','objectid'], axis=1).rename(columns={'idmanzan': 'ID', 'COORD_LATITUD':'Latitud','COORD_LONGITUD':'Longitud' })
deman['ID'] = deman['ID'].astype(str).str[-10:]
deman.sort_values(by='ID',inplace=True)
deman=deman.reset_index()

##Demanda

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1Y5W60AJfebmBBHtI3vrBNQIDChhVeY00/edit#gid=1140132466'
df_dem = pd.read_csv(f"{url.replace('/edit#gid=', '/export?format=csv&gid=')}")
df_dem

#Distacias

In [ ]:
import pandas as pd

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rx5IESeIzV6BKRAi4QH2-2gZV8LTbBK0/edit#gid=1086669528"

csv_export_url = spreadsheet_url.replace("/edit#gid=", "/export?format=csv&gid=")

dist_ff = pd.read_csv(csv_export_url,header=None)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1kuidLyRNKRDQha0c3j7vOiKN19ObYzsS/edit#gid=1613718108"

csv_export_url = spreadsheet_url.replace("/edit#gid=", "/export?format=csv&gid=")

dist_fd = pd.read_csv(csv_export_url,header=None)

In [ ]:
dist_fd=dist_fd.replace(',', '.', regex=True).astype(float)

In [ ]:
dist_fd

In [ ]:
fil, col=dist_fd.shape
for i in range(col):
  for j in range(fil):
    dist_fd[i][j]= 1 if dist_fd[i][j]<=1.5 else 0
dist_fd

In [ ]:
recuperar= dist_fd

#Parametros

In [ ]:
N = len(df_dem)
print(N)
M = len(df_fac)
T = 5

cap = 500
q = df_fac['q'].to_numpy()
q_max = 5

a = dist_fd
h_1 = df_dem['Afectados'].to_numpy() #demanda
print(h_1)
print()
h = [[h_1[i] if t == 1 else 0
      for t in range(T + 1)] for i in range(N)]
print(h) #demanda en el tiempo T
d = dist_ff
w1 = 1000
w2 = 250
w3 = 1000
#q

In [ ]:
a=a.transpose()

In [ ]:
a

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
N=982
h_1=h_1[:982]

In [ ]:
num_demand_points = N
num_facilities = M
coverage_matrix = a[:32].to_numpy()
demand_matrix = h_1
print(num_demand_points,num_facilities,type(coverage_matrix),type(demand_matrix))

In [ ]:
print(M)
print(coverage_matrix.shape)
print(N)
print(len(h_1))

In [ ]:
import random

In [ ]:
import random
import matplotlib.pyplot as plt
import networkx as nx
from math import sqrt

In [ ]:
def generador_data(fac, dem, deman_max):
    ancho = 2500
    cor_dem = [(random.randint(1, ancho), random.randint(1, ancho)) for _ in range(dem)]
    dem_p = [random.randint(1, deman_max) * 10 for _ in range(dem)]  # Tamaño de puntos escalado para visualización

    cor_fac = [(random.randint(1, ancho), random.randint(1, ancho)) for _ in range(fac)]

    # Crear el grafo
    G = nx.Graph()

    # Añadir nodos de demanda y facilidad al grafo
    for i, coord in enumerate(cor_dem):
        G.add_node(f"D{i}", pos=coord, demand=dem_p[i], color='red', size=dem_p[i])

    for j, coord in enumerate(cor_fac):
        G.add_node(f"F{j}", pos=coord, color='blue', size=100)  # Tamaño fijo para facilidades

    # Conectar nodos si cumplen una condición específica (e.g., todos conectados)
    cover=[]
    for i, c_dem in enumerate(cor_dem):
        for j, c_fac in enumerate(cor_fac):
            # Conectar si la distancia es menor a un umbral, ajustar según necesidad
            if sqrt((c_dem[0] - c_fac[0]) ** 2 + (c_dem[1] - c_fac[1]) ** 2) < 500:
                cover.append(1)
                G.add_edge(f"D{i}", f"F{j}")
            else:
                cover.append(0)

    cover=m_cover=pd.DataFrame(np.array(cover).reshape(fac,dem))
    # Establecer el tamaño de la figura (ancho x alto en pulgadas)
    plt.figure(figsize=(15, 15))  # Ajuste del tamaño de la figura a 15x15 pulgadas
    pos = nx.get_node_attributes(G, 'pos')
    colors = [G.nodes[node]['color'] for node in G.nodes]
    sizes = [G.nodes[node]['size'] for node in G.nodes]

    nx.draw(G, pos, node_color=colors, node_size=sizes, with_labels=False, edge_color='gray', alpha=0.6)
    plt.title('Red de Puntos de Demanda y Facilidades')
    plt.show()
    return dem_p,cover


In [ ]:
fac=33
demp=1000
deman_max=10
m_demp,m_cover=generador_data(fac, demp, deman_max)

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class HumanitarianFacilityLocationEnv(gym.Env):
    def __init__(self, num_demand_points, num_facilities, num_days, coverage_matrix, demand_matrix, max_installations_per_facility=5):
        super(HumanitarianFacilityLocationEnv, self).__init__()
        self.num_demand_points = num_demand_points
        self.num_facilities = num_facilities
        self.num_days = num_days
        self.current_day = 0
        self.move_duration = 1  # Días que tarda en trasladarse una instalación
        self.coverage_matrix = np.array(coverage_matrix, dtype=np.int32)
        self.demand_matrix = np.array(demand_matrix, dtype=np.int32)
        self.max_installations_per_facility = max_installations_per_facility

        # Inicializar el número de instalaciones por facility
        self.installations_per_facility = np.zeros(num_facilities, dtype=int)

        # Espacio de acción: mover una instalación a una nueva ubicación (discretizado)
        # Debes tener algo como esto:
        self.action_space = spaces.MultiDiscrete([2, num_facilities, 100, 100])

        # Espacio de observación: ubicaciones de instalaciones y puntos de demanda
        self.observation_space = spaces.Box(low=0, high=100, shape=(self.num_facilities + self.num_demand_points, 2), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            np.random.seed(seed)
        self.facility_locations = np.random.randint(0, 100, size=(self.num_facilities, 2))  # 100 es un valor arbitrario para coordenadas x e y
        self.demand_point_locations = np.random.randint(0, 100, size=(self.num_demand_points, 2))
        self.state = np.zeros((self.num_facilities + self.num_demand_points, 2), dtype=np.float32)
        self.state[:self.num_facilities] = self.facility_locations
        self.state[self.num_facilities:] = self.demand_point_locations
        self.current_day = 0
        self.move_end_day = np.full(self.num_facilities, -self.move_duration, dtype=int)  # Inicializar con días negativos
        self.installations_per_facility.fill(0)  # Reiniciar el número de instalaciones por facility
        return self.state, {}

    def step(self, action):
        action_type, facility_index, new_x, new_y = action

        if action_type == 0:  # Colocar nueva instalación
            if self.installations_per_facility[facility_index] < self.max_installations_per_facility:
                self.facility_locations[facility_index] = [new_x, new_y]
                self.installations_per_facility[facility_index] += 1
                self.move_end_day[facility_index] = self.current_day + self.move_duration

        elif action_type == 1:  # Mover instalación existente
            if self.current_day > self.move_end_day[facility_index] and self.installations_per_facility[facility_index] < self.max_installations_per_facility:
                self.facility_locations[facility_index] = [new_x, new_y]
                self.move_end_day[facility_index] = self.current_day + self.move_duration

        self.current_day += 1
        self.state[:self.num_facilities] = self.facility_locations
        self.state[self.num_facilities:] = self.demand_point_locations
        total_cost = self._calculate_total_cost()
        reward = -total_cost
        terminated = self.current_day == self.num_days - 1
        truncated = False
        info = {"total_cost": total_cost}  # Agrega el costo total al diccionario info
        return self.state, reward, terminated, truncated, info


    def _calculate_total_cost(self):
        total_cost = 0
        for day in range(self.current_day + 1):
            for i in range(self.num_demand_points):
                # Filtrar las instalaciones que están operativas en el día actual
                operational_facilities = np.where(self.move_end_day <= day)[0]
                if operational_facilities.size > 0:
                    # Verificar si al menos una instalación operativa cubre el punto de demanda i
                    covered = np.any(self.coverage_matrix[operational_facilities, i] == 1)
                    if covered:
                        # Sumar el costo de satisfacer la demanda
                        total_cost += self.demand_matrix[i]
                    else:
                        # Penalizar si ningún facility cubre este punto de demanda
                        total_cost += 1000 * self.demand_matrix[i]  # Penalización por demanda no atendida
                else:
                    # Penalizar si no hay instalaciones operativas
                    total_cost += 1000 * self.demand_matrix[i]

        # Añadir costos de abrir y de instalar centros de distribución
        num_opened_facilities = np.sum(self.installations_per_facility > 0)
        num_installed_centers = np.sum(self.installations_per_facility)
        total_cost += num_opened_facilities * 250  # Costo de abrir centros de distribución
        total_cost += num_installed_centers * 1000  # Costo de instalar centros de distribución

        return total_cost

In [ ]:
class CostLoggingCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(CostLoggingCallback, self).__init__(verbose)

    def _on_step(self):
        # Extraer 'infos' del entorno, que es donde se almacena el retorno de 'info' de 'step()'
        infos = self.locals["infos"]
        if infos and 'total_cost' in infos[0]:
            total_cost = infos[0]['total_cost']
            self.logger.record('total_cost', total_cost)
        return True

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

class TrainingProgressCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(TrainingProgressCallback, self).__init__(verbose)
        self.n_steps = 0

    def _on_step(self) -> bool:
        self.n_steps += 1
        if self.n_steps % 100 == 0:  # Imprime cada 100 steps
            print(f"Step number: {self.n_steps}")
        return True

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback
import numpy as np
import os

# Definir las matrices de cobertura y demanda
num_demand_points = demp
num_facilities = fac
coverage_matrix = m_cover
demand_matrix = m_demp

# Crear el entorno
env = HumanitarianFacilityLocationEnv(
    num_demand_points=num_demand_points,
    num_facilities=num_facilities,
    num_days=10,
    coverage_matrix=coverage_matrix,
    demand_matrix=demand_matrix,
    max_installations_per_facility=5
)

# Envolver el entorno en DummyVecEnv y Monitor
env = DummyVecEnv([lambda: Monitor(env)])

# Configurar el registro para TensorBoard
log_dir = "./ppo_humanitarian_facility_location_logs/"
os.makedirs(log_dir, exist_ok=True)

# Crear el modelo PPO con soporte para TensorBoard
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_dir)

# Configurar los callbacks
eval_callback = EvalCallback(env, best_model_save_path=log_dir,
                             log_path=log_dir, eval_freq=500,
                             deterministic=True, render=False)
progress_callback = TrainingProgressCallback(verbose=1)

# Entrenar el modelo con los callbacks
cost_callback = CostLoggingCallback()

model.learn(total_timesteps=10000, callback=[eval_callback, progress_callback, cost_callback])


# Guardar el modelo entrenado
model.save("ppo_humanitarian_facility_location")


In [ ]:
# Iniciar TensorBoard
%load_ext tensorboard
%tensorboard --logdir ./ppo_humanitarian_facility_location_logs/